### Stockfish vs Stockfish

In [ ]:
import os
import datetime

import chess.pgn
from rich.table import Table
import autorootcwd
import chess.engine
from src.utils.chess_utils import ChessBoard


def stockfish_vs_stockfish(
    stockfish_path: str, output_data: str, cpu_nums: int, num_games: int, compute_time: float
):
    """Play a game between two Stockfish instances.

    Args:
        stockfish_path (str): Path to the Stockfish executable.
        output_data (str): Path to the directory where the PGN files will be saved.
        cpu_nums (int, optional): Number of CPUs to be used by Stockfish. Defaults to 4.
        num_games (int, optional): Number of games to be played. Defaults to 1.
        compute_time (float, optional): Time in seconds each engine has to make a move. Defaults to 5.0.

    Returns:
        Game: A chess.pgn.Game object containing the game moves and results.
    """
    os.makedirs(output_data, exist_ok=True)
    for i in range(num_games):
        board = chess.Board()
        engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

        # Configure Stockfish to use more threads for increased speed
        engine.configure({"Threads": cpu_nums})

        game = chess.pgn.Game()
        node = game

        while not board.is_game_over():
            result = engine.play(board, chess.engine.Limit(time=compute_time))
            move = result.move
            board.push(move)
            ChessBoard().show(board, False)

            node = node.add_variation(move)

        engine.quit()

        # Save the game result to the PGN's headers
        game.headers["Result"] = board.result()
        game.headers["White"] = "Stockfish"
        game.headers["Black"] = "Stockfish"

        # Save to a PGN file
        current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        with open(f"{output_data}/stockfish_vs_stockfish_{current_time}.pgn", "w") as pgn_file:
            exporter = chess.pgn.FileExporter(pgn_file)
            game.accept(exporter)
        # free up memory
        del game
        del board


if __name__ == "__main__":
    stockfish_path = "./engine/stockfish/stockfish-ubuntu-x86-64-avx2"
    output_data = "./data/stockfish_vs_stockfish"
    cpu_nums = 4
    num_games = 1
    compute_time = 10.0
    stockfish_vs_stockfish(stockfish_path, output_data, cpu_nums, num_games, compute_time)

### lc0 vs lc0

In [ ]:
import os
import datetime

import chess.pgn
import autorootcwd
import chess.engine
from src.utils.chess_utils import ChessBoard


def lc0_vs_lc0(
    lc0_path: str, output_data: str, weights_file: str, num_games: int, compute_time: float
):
    """Play a game between two Lc0 instances.

    Args:
        lc0_path (str): Path to the Lc0 executable.
        output_data (str): Path to the directory where the PGN files will be saved.
        weights_file (str): Path to the weights file for Lc0.
        num_games (int): Number of games to be played. Defaults to 1.
        compute_time (float): Time in seconds each engine has to make a move. Defaults to 5.0.

    Returns:
        Game: A chess.pgn.Game object containing the game moves and results.
    """
    os.makedirs(output_data, exist_ok=True)
    for i in range(num_games):
        board = chess.Board()
        engine = chess.engine.SimpleEngine.popen_uci(lc0_path)

        # Configure Lc0 with the weights file
        engine.configure({"WeightsFile": weights_file})

        game = chess.pgn.Game()
        node = game

        limit = chess.engine.Limit(time=compute_time)

        while not board.is_game_over():
            result = engine.play(board, limit)
            move = result.move
            board.push(move)
            analysis_result = engine.analyse(board, limit)
            ChessBoard().show(board, False)

            node = node.add_variation(move)

        engine.send_game_result(board.result())

        engine.quit()

        # Save the game result to the PGN's headers
        game.headers["Result"] = board.result()
        game.headers["White"] = "Lc0"
        game.headers["Black"] = "Lc0"

        # Save to a PGN file
        current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        with open(f"{output_data}/lc0_vs_lc0_{current_time}.pgn", "w") as pgn_file:
            exporter = chess.pgn.FileExporter(pgn_file)
            game.accept(exporter)
        # free up memory
        del game
        del board


if __name__ == "__main__":
    lc0_path = "./engine/lc0/lc0"
    output_data = "./data/lc0_vs_lc0"
    weights_file = "./engine/lc0_model/t2-768x15x24h-swa-5230000.pb"
    num_games = 1
    compute_time = 1.0
    lc0_vs_lc0(lc0_path, output_data, weights_file, num_games, compute_time)

### lc0 vs Stockfish

In [ ]:
import os
import datetime

import chess.pgn
from rich.table import Table
import autorootcwd
from rich.console import Console
import chess.engine
from src.utils.chess_utils import ChessBoard

console = Console()


def lc0_vs_stockfish(
    lc0_path: str,
    output_data: str,
    cpu_nums: int,
    weights_file: str,
    stockfish_path: str,
    num_games: int,
    compute_time: float,
):
    os.makedirs(output_data, exist_ok=True)

    lc0_wins = 0
    stockfish_wins = 0
    draws = 0

    for i in range(num_games):
        board = chess.Board()
        lc0_engine = chess.engine.SimpleEngine.popen_uci(lc0_path)
        stockfish_engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

        # Configure Lc0 with the weights file
        lc0_engine.configure({"WeightsFile": weights_file})

        # Configure Stockfish to use more threads for increased speed
        stockfish_engine.configure({"Threads": 4})

        game = chess.pgn.Game()
        node = game

        while not board.is_game_over():
            if board.turn == chess.WHITE:
                result = lc0_engine.play(board, chess.engine.Limit(time=compute_time))
            else:
                result = stockfish_engine.play(board, chess.engine.Limit(time=compute_time))
            move = result.move
            board.push(move)
            ChessBoard().show(board, True)

            node = node.add_variation(move)

        lc0_engine.quit()
        stockfish_engine.quit()

        fen_state = board.fen()

        # Update win counters
        if board.result() == "1-0":
            lc0_wins += 1
        elif board.result() == "0-1":
            stockfish_wins += 1
        else:
            draws += 1

        # Save the game result to the PGN's headers
        game.headers["Result"] = board.result()
        game.headers["White"] = "Lc0"
        game.headers["Black"] = "Stockfish"

        # Save to a PGN file
        current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        with open(f"{output_data}/lc0_vs_stockfish_{current_time}.pgn", "w") as pgn_file:
            exporter = chess.pgn.FileExporter(pgn_file)
            game.accept(exporter)
        # free up memory
        del game
        del board

    lc0_winrate = (lc0_wins / num_games) * 100
    stockfish_winrate = (stockfish_wins / num_games) * 100
    drawrate = (draws / num_games) * 100

    table = Table(title="Lc0 vs Stockfish Results")
    table.add_column("Lc0 Winrate")
    table.add_column("Stockfish Winrate")
    table.add_column("Draw Rate")
    table.add_column("Total Games")
    table.add_row(
        f"{lc0_winrate}% ({lc0_winrate})",
        f"{stockfish_winrate}% ({stockfish_wins})",
        f"{drawrate}",
        f"{num_games}",
    )
    console.print(table)


if __name__ == "__main__":
    import autorootcwd

    lc0_path = "./engine/lc0/lc0"
    output_data = "data/lc0_vs_stockfish"
    cpu_nums = 4
    stockfish_path = "./engine/stockfish/stockfish-ubuntu-x86-64-avx2"
    weights_file = "./engine/lc0_model/t2-768x15x24h-swa-5230000.pb"
    num_games = 1
    compute_time = 10.0
    lc0_vs_stockfish(
        lc0_path, output_data, cpu_nums, weights_file, stockfish_path, num_games, compute_time
    )